Natalia Drzazga 

In [68]:
import pandas as pd
import re 
from datetime import timedelta


# wczytanie pliku billboard.csv
url="https://raw.githubusercontent.com/KAIR-ISZ/public_lectures/master/Analiza%20i%20Bazy%20Danych%202021/Lab%202/Datasets/billboard.csv"
data=pd.read_csv(url,encoding='latin1')

# zrobimy dwa pliczki, jeden z notowaniami a drugi z info o utworach
data_info = data[["year","artist.inverted","track","time","genre"]].copy()

# zamiana kolumn na wartości
data_melted = pd.melt(data, id_vars=["year","artist.inverted","track","time","genre","date.entered","date.peaked"],var_name="week_number", value_name="rank").dropna()
data_melted = data_melted.drop(['year','artist.inverted','time','genre'], 1)

# zrobienie porządku z tygodniami
unique_weeks = data_melted.week_number.unique()
for week in unique_weeks:
    week_number = int(''.join(re.findall(r'\d+', week)))
    data_melted = data_melted.replace(to_replace=week, value=week_number)
    
# zrobienie jednej obecnej daty    
data_melted['date.entered'] = pd.to_datetime(data_melted['date.entered'])
data_series=[]
for index, row in data_melted.iterrows():
    date = row['date.entered'] + timedelta(weeks=row['week_number']-1)
    data_series.append(date)

data_melted['date'] = data_series  
data_melted = data_melted.drop(['date.peaked','date.entered','week_number'], 1).sort_values(['track','date']).reset_index()

# zapisanie uporządkowanej tabeli
data_melted.to_csv('billboard_rank.csv')
data_info.to_csv('tracks_info.csv')

